In [14]:
# Import required libraries
from OpenSSL import crypto, SSL  # Main OpenSSL module for cryptography
from cryptography.hazmat.primitives import serialization  # Key serialization
from cryptography.hazmat.primitives.asymmetric import padding  # Encryption padding
import hashlib  # Hashing algorithms
import hmac  # Hash-based message authentication
import os  # Operating system functions
import base64  # Base64 encoding/decoding
import socket  # Network connections

In [ ]:
# ========================
# KEY MANAGEMENT FUNCTIONS
# ========================

def generate_rsa_key(key_size=2048):
    """
    Creates a new RSA encryption key pair (public + private key).
    
    Parameters:
    key_size (int): Key length in bits. Bigger = more secure but slower.
                    Common values: 1024 (weak), 2048 (standard), 4096 (strong)
    
    Returns:
    crypto.PKey: Contains both public and private RSA keys
    
    Example:
    >>> key = generate_rsa_key()
    >>> print(type(key))
    <class 'OpenSSL.crypto.PKey'>
    """
    key = crypto.PKey()  # Create empty key container
    key.generate_key(crypto.TYPE_RSA, key_size)  # Generate RSA keys
    return key

def save_private_key(key, filename, passphrase=None):
    """
    Saves a private key to a file, optionally password-protected.
    
    Parameters:
    key:        The private key from generate_rsa_key()
    filename:   Where to save the key (e.g., 'mykey.pem')
    passphrase: Optional password to encrypt the key file (None = no password)
    
    Example:
    >>> key = generate_rsa_key()
    >>> save_private_key(key, 'secret.key', 'my-password')
    """
    # Convert key to PEM format (standard text format)
    if passphrase:
        # If password provided, encrypt the key file
        pem = crypto.dump_privatekey(
            crypto.FILETYPE_PEM, 
            key, 
            passphrase=passphrase.encode()  # Convert password to bytes
        )
    else:
        # Unencrypted key
        pem = crypto.dump_privatekey(crypto.FILETYPE_PEM, key)
    
    # Write to file
    with open(filename, 'wb') as f:  # 'wb' = write binary mode
        f.write(pem)

def load_private_key(filename, passphrase=None):
    """
    Loads a private key from an encrypted or unencrypted file.
    
    Parameters:
    filename:   Path to key file (e.g., 'secret.key')
    passphrase: Password if the key file is encrypted
    
    Returns:
    crypto.PKey: Loaded private key
    
    Example:
    >>> key = load_private_key('secret.key', 'my-password')
    """
    with open(filename, 'rb') as f:  # 'rb' = read binary mode
        key_data = f.read()  # Read entire file
    
    # Load key with password handling
    return crypto.load_privatekey(
        crypto.FILETYPE_PEM,  # PEM file format
        key_data,
        passphrase=passphrase.encode() if passphrase else None
    )

# ==========================
# CERTIFICATE FUNCTIONS
# ==========================

def create_self_signed_cert(key, **subject):
    """
    Creates a self-signed SSL certificate (like websites use for HTTPS).
    
    Parameters:
    key:      Private key from generate_rsa_key()
    **subject: Certificate details as keyword arguments:
               - CN: Common Name (e.g., website domain)
               - O:  Organization
               - OU: Organizational Unit
               - C:  Country (2-letter code)
               - ST: State/Province
               - L:  Locality/City
    
    Returns:
    crypto.X509: Certificate object
    
    Example:
    >>> key = generate_rsa_key()
    >>> cert = create_self_signed_cert(key, CN='localhost', O='My Org')
    """
    cert = crypto.X509()  # Create empty certificate
    
    # Set certificate subject details
    cert_subject = cert.get_subject()
    for field, value in subject.items():
        setattr(cert_subject, field, value)  # e.g., cert_subject.CN = 'localhost'
    
    # Certificate identification
    cert.set_serial_number(1000)  # Must be unique per certificate
    cert.gmtime_adj_notBefore(0)  # Valid starting now
    cert.gmtime_adj_notAfter(365*24*60*60)  # Expires in 1 year
    
    # Self-signed = issuer is same as subject
    cert.set_issuer(cert_subject)
    cert.set_pubkey(key)  # Link to public key
    cert.sign(key, 'sha256')  # Sign with private key
    
    return cert

def save_cert(cert, filename):
    """
    Saves a certificate to a file.
    
    Parameters:
    cert:     Certificate from create_self_signed_cert()
    filename: Output file (e.g., 'mycert.pem')
    
    Example:
    >>> save_cert(cert, 'website_cert.pem')
    """
    with open(filename, 'wb') as f:
        f.write(crypto.dump_certificate(crypto.FILETYPE_PEM, cert))

# ========================
# ENCRYPTION/DECRYPTION
# ========================

def rsa_encrypt(public_key, data):
    """
    Encrypts data using an RSA public key.
    
    Parameters:
    public_key: The PUBLIC part of an RSA key pair
    data:       Bytes to encrypt (max length depends on key size)
    
    Returns:
    bytes: Encrypted data
    
    Example:
    >>> key = generate_rsa_key()
    >>> encrypted = rsa_encrypt(key, b'Secret Message')
    """
    return public_key.encrypt(
        data,
        padding.PKCS1v15()  # Standard padding scheme
    )

def rsa_decrypt(private_key, ciphertext):
    """
    Decrypts data using an RSA private key.
    
    Parameters:
    private_key: The PRIVATE part of RSA key pair
    ciphertext:  Encrypted data from rsa_encrypt()
    
    Returns:
    bytes: Original decrypted data
    
    Example:
    >>> decrypted = rsa_decrypt(key, encrypted)
    >>> print(decrypted)
    b'Secret Message'
    """
    return private_key.decrypt(
        ciphertext,
        padding.PKCS1v15()
    )

# ========================
# SSL/TLS FUNCTIONS
# ========================

def create_ssl_context(certfile, keyfile):
    """
    Creates SSL configuration for secure connections.
    
    Parameters:
    certfile: Path to certificate file
    keyfile:  Path to private key file
    
    Returns:
    SSL.Context: Configuration for SSL connections
    
    Example:
    >>> context = create_ssl_context('cert.pem', 'key.pem')
    """
    context = SSL.Context(SSL.TLSv1_2_METHOD)  # Use TLSv1.2 protocol
    context.use_certificate_file(certfile)  # Load certificate
    context.use_privatekey_file(keyfile)  # Load private key
    return context

# ========================
# HASHING & SIGNATURES
# ========================

def generate_hash(data, algorithm='sha256'):
    """
    Creates a cryptographic fingerprint of data.
    
    Parameters:
    data:      Bytes to hash (e.g., file contents)
    algorithm: Hash algorithm (sha256, md5, sha1, etc.)
    
    Returns:
    bytes: Unique hash value (digest)
    
    Example:
    >>> hash_value = generate_hash(b'Hello World')
    >>> print(hash_value.hex())
    a591a6d40bf420...
    """
    h = hashlib.new(algorithm)  # Create hash object
    h.update(data)  # Feed data to hash function
    return h.digest()  # Get final hash value

# ========================
# CERTIFICATE UTILITIES
# ========================

def get_cert_info(cert):
    """
    Extracts basic information from a certificate.
    
    Parameters:
    cert: Certificate object
    
    Returns:
    dict: Certificate details
    
    Example:
    >>> info = get_cert_info(cert)
    >>> print(info['subject'])
    'localhost'
    """
    return {
        'subject': cert.get_subject().CN,  # Common Name
        'issuer': cert.get_issuer().CN,
        'expiry': cert.get_notAfter().decode(),  # Expiration date
        'serial': hex(cert.get_serial_number())  # Unique ID
    }

# ========================
# NETWORK SECURITY
# ========================

def create_self_signed_tls_pair():
    """
    Generates ready-to-use TLS certificate and key files.
    
    Returns:
    tuple: (certificate_file, key_file)
    
    Example:
    >>> cert_file, key_file = create_self_signed_tls_pair()
    >>> print(cert_file, key_file)
    ('cert.pem', 'key.pem')
    """
    key = generate_rsa_key()
    cert = create_self_signed_cert(key, CN='localhost')
    save_private_key(key, 'key.pem')
    save_cert(cert, 'cert.pem')
    return 'cert.pem', 'key.pem'


In [ ]:
generate_rsa_key()

In [ ]:
save_private_key()

In [ ]:
load_private_key()

In [ ]:
create_self_signed_cert()

In [ ]:
save_cert()

In [ ]:
rsa_encrypt()

In [ ]:
rsa_decrypt()

In [ ]:
create_ssl_context()

In [ ]:
generate_hash()

In [ ]:
get_cert_info()

In [ ]:
create_self_signed_tls_pair()